In [ ]:
import joblib

In [ ]:
import pandas as pd

In [ ]:
df1=pd.read_csv('/Users/user2/Desktop/Code/Bihar_model/Bihar Assembly Election - AE.csv')

In [ ]:
loaded_catboost_model = joblib.load('catboost_model.pkl')
loaded_logistic_model = joblib.load('logistic_model.pkl')
loaded_scaler = joblib.load('scaler.pkl')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.3.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
def predict_votes_and_win_prob(ac_no, party, alliance):
    # Predict normalized votes using CatBoost
    input_data = pd.DataFrame({
        'AC No': [ac_no],
        'Party': [party],
        'Alliance': [alliance]
    })
    
    # Predict normalized votes
    predicted_normalized_votes = loaded_catboost_model.predict(input_data)
    
    # Factor for valid votes
    factor = df1[(df1['AC No'] == ac_no) & (df1['Year'] == 2020)]['Valid Votes'].values[0] * 1.1
    predicted_vote_count = round(predicted_normalized_votes[0] * factor)
    
    # Use the predicted normalized votes in Logistic Regression to predict win probability
    normalized_votes_scaled = loaded_scaler.transform([[predicted_normalized_votes[0]]])
    win_prob = loaded_logistic_model.predict_proba(normalized_votes_scaled)
    
    # Return both predicted normalized votes (adjusted) and win probabilities
    return {
        'Predicted Normalized Votes (adjusted)': predicted_vote_count,
        'Prob of Loss (Win=0)': win_prob[0][0],
        'Prob of Win (Win=1)': win_prob[0][1]
    }

future_ac_no = 1
future_party = 'JD(U)'
alliance = 'NDA'

predictions = predict_votes_and_win_prob(future_ac_no, future_party, alliance)
print(predictions)

{'Predicted Normalized Votes (adjusted)': 72301, 'Prob of Loss (Win=0)': 0.683923092052698, 'Prob of Win (Win=1)': 0.316076907947302}


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
